In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [2]:
#importing different python Libraries  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import seaborn as sns

In [3]:
import tensorflow as tf
from tensorflow.keras import layers,models,datasets

In [4]:
import tensorflow as tf
from tensorflow import keras

from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Activation,BatchNormalization,GlobalAveragePooling2D,Flatten
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator


In [5]:
!pip install py7zr

     |████████████████████████████████| 69 kB 818 kB/s            
     |████████████████████████████████| 126 kB 1.5 MB/s            
     |████████████████████████████████| 48 kB 3.2 MB/s             
     |████████████████████████████████| 2.4 MB 6.7 MB/s            
     |████████████████████████████████| 357 kB 63.0 MB/s            
     |████████████████████████████████| 43 kB 1.2 MB/s             
     |████████████████████████████████| 2.0 MB 56.2 MB/s            


In [6]:
from py7zr import unpack_7zarchive
import shutil

In [7]:
shutil.register_unpack_format('7zip',['.7z'],unpack_7zarchive)

In [8]:
#unpack train images to temp directory
shutil.unpack_archive('/kaggle/input/cifar-10/train.7z','/kaggle/temp/')

In [9]:
train_labels = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv', header= 'infer')

#unique labels
classes = train_labels['label'].unique()

print(classes)

['frog' 'truck' 'deer' 'automobile' 'bird' 'horse' 'ship' 'cat' 'dog'
 'airplane']


In [10]:
import os

if not os.path.exists("/kaggle/temp/validation/"):
    os.mkdir("/kaggle/temp/validation/")

parent_path_train = "/kaggle/temp/train/"
parent_path_validation = "/kaggle/temp/validation/"
parent_path_test = "/kaggle/temp/test/"


for class1 in classes:
    path_train = os.path.join(parent_path_train,class1)
    if not os.path.exists(path_train):
        os.mkdir(path_train)
        
    path_validation = os.path.join(parent_path_validation,class1)
    if not os.path.exists(path_validation):
        os.mkdir(path_validation)
        
for (int_ind,row) in train_labels.iterrows():
    id = str(row["id"]) + ".png"
    source_path = os.path.join(parent_path_train,id)
    
    p = np.random.random()
    if p <= 0.80:
        target_path = os.path.join(parent_path_train,row["label"],id)
        os.replace(source_path,target_path)
    else:
        target_path = os.path.join(parent_path_validation,row["label"],id)
        os.replace(source_path,target_path)


In [11]:
def my_model():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='valid',activation=None,use_bias=False,input_shape=(32,32,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=48, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=80, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(units=10))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = my_model()





2022-03-22 15:46:02.132325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 15:46:02.212105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 15:46:02.212834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 15:46:02.214186: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [12]:

reduceLROnPlateau = ReduceLROnPlateau(monitor='val_acc',
                                      patience=3,
                                     verbose=1,
                                     factor=0.5,
                                     min_lr=0.00001)

In [13]:
train_datagen = ImageDataGenerator(featurewise_center= False,
                                  samplewise_center= False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,
                                  zoom_range=0.1,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  rescale=1./255
                                  )

In [14]:

train_gen = train_datagen.flow_from_directory(directory='/kaggle/temp/train/',target_size=(32,32),batch_size=128)
validation_datagen = ImageDataGenerator(rescale=1./255)
valid_gen = validation_datagen.flow_from_directory(directory='/kaggle/temp/validation/',target_size=(32,32),batch_size=128)



Found 39894 images belonging to 10 classes.
Found 10106 images belonging to 10 classes.


In [15]:
model.fit(train_gen,epochs=50,callbacks=[reduceLROnPlateau],validation_data=valid_gen,steps_per_epoch=train_gen.n//train_gen.batch_size,
         validation_steps= valid_gen.n//valid_gen.batch_size,workers=8,use_multiprocessing=True)

2022-03-22 15:46:06.466189: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-03-22 15:46:09.503350: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


311/311 [==============================] - 40s 100ms/step - loss: 1.4821 - accuracy: 0.4917 - val_loss: 2.4521 - val_accuracy: 0.2008
Epoch 2/50
311/311 [==============================] - 32s 101ms/step - loss: 1.1646 - accuracy: 0.6172 - val_loss: 1.4894 - val_accuracy: 0.5250
Epoch 3/50
311/311 [==============================] - 32s 99ms/step - loss: 1.0031 - accuracy: 0.6745 - val_loss: 1.0319 - val_accuracy: 0.6569
Epoch 4/50
311/311 [==============================] - 32s 99ms/step - loss: 0.8995 - accuracy: 0.7043 - val_loss: 0.9856 - val_accuracy: 0.6703
Epoch 5/50
311/311 [==============================] - 31s 98ms/step - loss: 0.8245 - accuracy: 0.7283 - val_loss: 1.1823 - val_accuracy: 0.6060
Epoch 6/50
311/311 [==============================] - 34s 107ms/step - loss: 0.7696 - accuracy: 0.7450 - val_loss: 1.0095 - val_accuracy: 0.6608
Epoch 7/50
311/311 [==============================] - 33s 102ms/step - loss: 0.7251 - accuracy: 0.7597 - val_loss: 0.9171 - val_accuracy: 0.6944

In [16]:
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z','/kaggle/temp/test')

In [17]:
shutil.unregister_unpack_format('7zip')
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(directory='/kaggle/temp/test',target_size=(32,32),batch_size=64,class_mode=None,shuffle=False)

Found 300000 images belonging to 1 classes.


In [18]:
test_gen.reset()
predictions_vectors = model.predict(test_gen)


predictions_final=np.argmax(predictions_vectors, axis=1)
print(predictions_final)

[9 6 2 ... 3 5 6]


In [19]:
print(type(train_gen.class_indices))
print(train_gen.class_indices)

classes = {value:key for (key,value) in train_gen.class_indices.items()}
print(classes)

<class 'dict'>
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [20]:
predicted_classes=np.empty(shape=300000,dtype=np.dtype('U20'))


In [21]:

ind=0
for i in predictions_final.tolist():
    predicted_classes[ind]=classes[i]
    ind=ind+1
    


In [22]:
filenames_wo_ext = []
for fname in test_gen.filenames:
    filenames_wo_ext.append(int(fname.split(sep="/")[1].split(sep=".")[0])-1)



In [23]:
predicted_classes_final = np.empty(shape=300000,dtype=np.dtype('U20'))
predicted_classes_final[filenames_wo_ext]=predicted_classes


In [24]:
sub = pd.read_csv('../input/cifar-10/sampleSubmission.csv',header='infer')
sub.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      300000 non-null  int64 
 1   label   300000 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [25]:
sub['label'] = predicted_classes_final
sub.to_csv('submission.csv',index=False)